#### Частина 1. Data Cleaning

In [4]:
import numpy as np 
import pandas as pd 

import matplotlib.pyplot as plt 
import seaborn as sns 

import random
random.seed(42) 

import warnings
warnings.filterwarnings('ignore')

import os

In [ ]:
data_path = "you path"
if os.path.exists(data_path):
    df = pd.read_csv(data_path)
else:
    raise FileNotFound(f'File {data_path} not found')

True


In [3]:
df.head()

,id,time,con_treat,page,converted
0,851104,11:48.6,control,old_page,0
1,804228,01:45.2,control,old_page,0
2,661590,55:06.2,treatment,new_page,0
3,853541,28:03.1,treatment,new_page,0
4,864975,52:26.2,control,old_page,1


In [ ]:
df.columns = ["user_id", "timestamp", "group", "landing_page", "converted"]
df.head()

In [ ]:
print(f'Number of rows: {df.shape[0]}')
print(f'Number of unique users: {df.user_id.nunique()}')

In [ ]:
# Чи збігаються номери new_page та treatment?
n_treat = df[df["group"] == "treatment"].shape[0]
n_new_page = df[df["landing_page"] == "new_page"].shape[0]
difference = n_treat - n_new_page

pd.DataFrame({
    'N treatment': [n_treat],
    'N new_page': [n_new_page],
    'Difference': [difference]
})

In [ ]:
# unmatch 
df[(df["group"] == "treatment") & (df["landing_page"] == "old_page")]

In [ ]:
df_mismatch = df[(df["group"] == "treatment") & (df["landing_page"] == "old_page")
               |(df["group"] == "control") & (df["landing_page"] == "new_page")]

n_mismatch = df_mismatch.shape[0]

percent_mismatch = round(n_mismatch / len(df) * 100, 2)
print(f'Number of mismatched rows: {n_mismatch} rows')
print(f'Percent of mismatched rows: {percent_mismatch} percent')

In [ ]:
df2 = df[(df["group"] == "treatment") & (df["landing_page"] == "new_page")
        |(df["group"] == "control") & (df["landing_page"] == "old_page")]

len(df2)

In [ ]:
df_mismatch = df2[(df2["group"] == "treatment") & (df2["landing_page"] == "old_page")
               |(df2["group"] == "control") & (df2["landing_page"] == "new_page")]

n_mismatch = df_mismatch.shape[0]

percent_mismatch = round(n_mismatch / len(df2) * 100, 2)
print(f'Number of mismatched rows: {n_mismatch} rows')
print(f'Percent of mismatched rows: {percent_mismatch} percent')

In [ ]:
df2[df2.duplicated("user_id") == True]

#### Частина 2. Probability

In [ ]:
df2.converted.mean() * 100

In [ ]:
df2.user_id = df2.user_id.astype(str)
df2.groupby("group").mean() * 100

In [ ]:
pd.DataFrame(df2.landing_page.value_counts(normalize = True) * 100)

> Чи є достатньо доказів, щоб зробити висновок, що нова сторінка лікування призводить до більшої кількості конверсій? \
> Імовірність того, що особа отримала нову сторінку, становить 50%. \
> Імовірність того, що особа здійснить конверсію незалежно від сторінки, яку вона отримала, становить 11,96%. \
> Враховуючи, що особа перебувала в контрольній групі, ймовірність її конверсії становить 12,04%. \
> Враховуючи, що особа перебувала в групі лікування, ймовірність її конверсії становить 11,88%. \
> 1 до 4 свідчить про те, що між групою лікування та контрольною групою немає істотної різниці в конверсії. Отже, ми можемо зробити висновок, що нова сторінка лікування не має впливу і не призводить до збільшення кількості конверсій.


#### Частина 3. A/B Test

In [ ]:
means_diff = []
size = df.shape[0]
for _ in range(10000):
    sample = df2.sample(size, replace = True)
    control_mean = sample[sample["group"] == "control"]["converted"].mean()
    treat_mean = sample[sample["group"] == "treatment"]["converted"].mean()
    means_diff.append(treat_mean - control_mean)

In [ ]:
plt.figure(figsize = (8,4), dpi = 100)
plt.hist(means_diff, bins = 25)
plt.show()

In [ ]:
plt.figure(figsize = (8,4), dpi = 100)
plt.hist(null_vals, bins = 25)
plt.show()

In [ ]:
control_mean = df2[df2["group"] == "control"]["converted"].mean()
treat_mean = df2[df2["group"] == "treatment"]["converted"].mean()
obs_diff = treat_mean - control_mean

plt.figure(figsize = (8,4), dpi = 100)
plt.hist(null_vals, bins = 25)
plt.axvline(obs_diff, c='red')
plt.show()

In [ ]:
(null_vals > obs_diff).mean()

In [ ]:
pd.crosstab(df_2["group"], df_2["landing_page"])


In [ ]:
conv = df_2.groupby("group")["converted"].agg(["sum", "count"])
conv


In [ ]:
control = df_2[df_2.group == "control"]["converted"]
treat   = df_2[df_2.group == "treatment"]["converted"]


In [ ]:
n_c = len(control)
n_t = len(treat)

x_c = control.sum()
x_t = treat.sum()

In [ ]:
p_c = x_c / n_c
p_t = x_t / n_t

p_c, p_t, p_t - p_c


In [ ]:
from scipy.stats import norm


p_pool = (x_c + x_t) / (n_c + n_t)


se = np.sqrt(p_pool * (1 - p_pool) * (1/n_c + 1/n_t))


z = (p_t - p_c) / se

p_value = 2 * (1 - norm.cdf(abs(z)))

z, p_value


In [ ]:

table = np.array([
    [control.sum(), control.count() - control.sum()],
    [treat.sum(),   treat.count()   - treat.sum()]
])

table

In [ ]:
# Chi2
from scipy.stats import chi2_contingency

chi2, p_value, dof, expected = chi2_contingency(table)
chi2, p_value


In [ ]:
# Fisher 
from scipy.stats import fisher_exact

odds_ratio, p_value = fisher_exact(table)
odds_ratio, p_value


In [ ]:
mde = (z_alpha + z_beta) * np.sqrt(2 * p0 * (1 - p0) / n)

mde

In [ ]:
real_uplift = p_t - p_c

real_uplift, mde


#### Data Regression (after lessons)

In [ ]:
df2["intercept"] = 1
df2.head()